In [1]:
train_url = 'https://raw.githubusercontent.com/sidt-ai/data-science-competitions/main/machinehack/ode_to_code_2022/data/raw/train.csv'
test_url = 'https://raw.githubusercontent.com/sidt-ai/data-science-competitions/main/machinehack/ode_to_code_2022/data/raw/test.csv'
sub_url = 'https://raw.githubusercontent.com/sidt-ai/data-science-competitions/main/machinehack/ode_to_code_2022/data/raw/sample_submission.csv'

In [2]:
import gc
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

In [3]:
train = pd.read_csv(train_url)

test = pd.read_csv(test_url)
test.drop(['season'], axis=1, inplace=True)

sub = pd.read_csv(sub_url)

# AutoGluon (Model 2)

In [4]:
!python3 -m pip install --quiet pip
!python3 -m pip install --quiet setuptools wheel
!python3 -m pip install --quiet "mxnet<2.0.0"
!python3 -m pip install --quiet autogluon

     |████████████████████████████████| 47.3 MB 1.9 MB/s 
     |████████████████████████████████| 352 kB 8.8 MB/s 
     |████████████████████████████████| 56 kB 162 kB/s 
     |████████████████████████████████| 52 kB 1.6 MB/s 
     |████████████████████████████████| 273 kB 68.6 MB/s 
     |████████████████████████████████| 22.3 MB 1.4 MB/s 
     |████████████████████████████████| 27.4 MB 1.4 MB/s 
     |████████████████████████████████| 210 kB 70.3 MB/s 
     |████████████████████████████████| 822 kB 55.5 MB/s 
     |████████████████████████████████| 131 kB 80.7 MB/s 
     |████████████████████████████████| 4.2 MB 35.9 MB/s 
     |████████████████████████████████| 119 kB 71.9 MB/s 
     |████████████████████████████████| 1.3 MB 60.7 MB/s 
     |████████████████████████████████| 3.0 MB 53.2 MB/s 
     |████████████████████████████████| 296 kB 53.4 MB/s 
     |████████████████████████████████| 166.7 MB 17 kB/s 
     |████████████████████████████████| 189 kB 65.9 MB/s 
     |█████████████

In [5]:
from autogluon.tabular import TabularPredictor

In [6]:
label = 'season'
eval_metric = 'accuracy'
save_path = '/AutoGluons/'

predictor = TabularPredictor(label=label, eval_metric=eval_metric, 
                             path=save_path, verbosity=2)
predictor.fit(train, presets='best_quality', time_limit=600)

results = predictor.fit_summary()

Presets specified: ['best_quality']
Beginning AutoGluon training ... Time limit = 600s
AutoGluon will save models to "/AutoGluons/"
AutoGluon Version:  0.3.1
Train Data Rows:    42748
Train Data Columns: 13
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == object).
	4 unique label values:  ['a', 's', 'u', 'w']
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
NumExpr defaulting to 2 threads.
Train Data Class Count: 4
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    12712.07 MB
	Train Data (Original)  Memory Usage: 25.59 MB (0.2% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators

*** Summary of fit() ***
Estimated performance of each model:
                    model  score_val  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0     WeightedEnsemble_L2   0.528539       1.825294  397.458472                0.005249           3.711689            2       True          5
1  NeuralNetFastAI_BAG_L1   0.527838       0.896321  376.853164                0.896321         376.853164            1       True          3
2     WeightedEnsemble_L3   0.524258       3.476197  591.133706                0.006577           3.204824            3       True          8
3  NeuralNetFastAI_BAG_L2   0.524212       3.212845  579.755080                1.177693         185.930678            2       True          6
4         LightGBM_BAG_L1   0.521685       0.710322   16.812191                0.710322          16.812191            1       True          4
5       LightGBMXT_BAG_L2   0.513381       2.291927  401.998204                0.25677

In [7]:
preds = predictor.predict(test)

In [8]:
sub2 = sub.copy()
sub2['season'] = preds
sub2.to_csv('sub2.csv', index=False)

In [9]:
# del predictor
# gc.collect()

# H20 AutoML (Model 3)

In [10]:
!pip install --quiet --upgrade h2o

     |████████████████████████████████| 175.8 MB 30 kB/s 


In [11]:
import h2o
from h2o.automl import H2OAutoML

In [12]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.13" 2021-10-19; OpenJDK Runtime Environment (build 11.0.13+8-Ubuntu-0ubuntu1.18.04); OpenJDK 64-Bit Server VM (build 11.0.13+8-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmprddhu4uf
  JVM stdout: /tmp/tmprddhu4uf/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmprddhu4uf/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.7
H2O_cluster_version_age:,27 days
H2O_cluster_name:,H2O_from_python_unknownUser_cef9ys
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [13]:
h2o_train = h2o.H2OFrame(train)
h2o_test = h2o.H2OFrame(test)

h2o_train['season'] = h2o.H2OFrame(train['season'].to_numpy().ravel()).asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [14]:
features = [f for f in test.columns]

In [15]:
model3 = H2OAutoML(stopping_metric='misclassification', max_runtime_secs=600)
model3.train(x=features, y='season', training_frame=h2o_train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_1_AutoML_1_20220118_81554

No model summary for this model

ModelMetricsMultinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.2498041524871099
RMSE: 0.4998041141158303

ModelMetricsMultinomialGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.2802817622030454
RMSE: 0.5294164355241018


In [16]:
model3.leaderboard

model_id,mean_per_class_error,logloss,rmse,mse
StackedEnsemble_BestOfFamily_1_AutoML_1_20220118_81554,0.515821,0.75375,0.529416,0.280282
XGBoost_1_AutoML_1_20220118_81554,0.547336,0.788386,0.541153,0.292846
GBM_1_AutoML_1_20220118_81554,0.584124,0.790964,0.546989,0.299197
GLM_1_AutoML_1_20220118_81554,0.634337,0.921559,0.580257,0.336698


In [17]:
preds3 = model3.leader.predict(h2o_test).as_data_frame()['predict']

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [18]:
sub3 = sub.copy()
sub3['season'] = preds3
sub3.to_csv('sub3.csv', index=False)

In [19]:
# del model3
# gc.collect()

# MLJAR (Model 4)

In [20]:
# !pip install --quiet --uprade mljar-supervised

In [21]:
# from supervised.automl import AutoML
# from sklearn.metrics import accuracy_score

In [22]:
# model4 = AutoML(mode='compete', 
#                 ml_task='multiclass_classification', 
#                 eval_metric='accuracy', 
#                 total_time_limit=600)

# model4.fit(X=train[features], y=train['season'])

In [23]:
# preds4 = model4.predict(test)

In [24]:
# sub4 = sub.copy()
# sub4['season'] = preds4
# sub4.to_csv('sub4.csv', index=False)

In [25]:
# del model4
# gc.collect()

# FLAML (Model 5)

In [51]:
!pip install --quiet --upgrade flaml

     |████████████████████████████████| 142 kB 5.5 MB/s 
     |████████████████████████████████| 157.5 MB 51 kB/s 


In [52]:
from flaml import AutoML

In [53]:
model5 = AutoML()
model5.fit(train[test.columns], train['season'], 
           task='classification', time_budget=600)

[flaml.automl: 01-18 08:51:52] {2007} INFO - task = classification
task = classification
[flaml.automl: 01-18 08:51:52] {2009} INFO - Data split method: stratified
Data split method: stratified
[flaml.automl: 01-18 08:51:52] {2013} INFO - Evaluation method: cv
Evaluation method: cv
[flaml.automl: 01-18 08:51:52] {2113} INFO - Minimizing error metric: log_loss
Minimizing error metric: log_loss
[flaml.automl: 01-18 08:51:52] {2170} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'catboost', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'lrl1']
List of ML learners in AutoML Run: ['lgbm', 'rf', 'catboost', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'lrl1']
[flaml.automl: 01-18 08:51:52] {2437} INFO - iteration 0, current learner lgbm
iteration 0, current learner lgbm
[flaml.automl: 01-18 08:51:53] {2551} INFO - Estimated sufficient time budget=9075s. Estimated necessary time budget=223s.
Estimated sufficient time budget=9075s. Estimated necessary time budget=223s.
[flaml.automl: 01-1

In [54]:
preds5 = model5.predict(test)

In [55]:
sub5 = sub.copy()
sub5['season'] = preds5
sub5.to_csv('sub5.csv', index=False)